In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix, issparse
from scipy.sparse.linalg import norm as spnorm
from ntf_cython.nmf import NMF as NMF_BPP
from ntf_cython.nmf import bpp as bpp
from ntf_cython.random import algo42
import scipy.optimize as optimize
from sklearn.decomposition.nmf import _initialize_nmf
import pickle
import sklearn
from scipy import sparse

%matplotlib inline

In [3]:
%%time
df = pd.read_csv('/Users/user/Documents/Medicare_Provider_Util_Payment_PUF_CY2013/Medicare_Provider_Util_Payment_PUF_CY2013.txt', sep= '\t')

<string>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 1min 6s, sys: 12.5 s, total: 1min 19s
Wall time: 1min 23s


In [4]:
#Clean up first row of the data
df = df[1:]
np.shape(df)

(9287876, 28)

In [5]:
%%time
row_name = 'NPI'
col_name = 'HCPCS_CODE'
measure = 'AVERAGE_SUBMITTED_CHRG_AMT'
# filters: averages are calculated by NPI, HCPCS_CODE and PLACE_OF_SERVICE
# so need to fix PLACE_OF_SERVICE to not be forced to average averages.
# The rest of the filters are up to you.
filters = (df.PLACE_OF_SERVICE == 'F')
filters &= (df.NPPES_ENTITY_CODE == 'O')
filters &= (df.NPPES_PROVIDER_COUNTRY == 'US')
# data = df[filters][[row_name, col_name, measure]].dropna().set_index([row_name, col_name])
data = df[filters].dropna(subset=[measure]).groupby([row_name, col_name])[measure].max()
idx_row = data.index.names.index(row_name)
idx_col = data.index.names.index(col_name)
M = coo_matrix((data.values,
                (data.index.labels[idx_row], data.index.labels[idx_col])),
               shape=[len(data.index.levels[idx_row]), len(data.index.levels[idx_col])]).astype(float)
print(M.shape)

(14822, 1376)
CPU times: user 4.05 s, sys: 1.03 s, total: 5.08 s
Wall time: 5.21 s


In [6]:
def reconstruction_error(M, WH):
    """
    This function computes the relative error of a sparse matrix. By concept, this computation is almost the same with
    computing the reconstruction error of the factor matrices formed using python non-negative matrix factorization
    
    Accepts
    -------
    M  : array-like
    WH : the product of the loading matrix W and the coefficient matrix H
    
    Returns
    -------
    a float-value corresponding to the reconstruction error
    """
    
    nonzero_row, nonzero_col = M.nonzero()
    orig = M.todense()[nonzero_row, nonzero_col]
    reconstructed = WH[nonzero_row, nonzero_col]
    
    return np.linalg.norm(orig - reconstructed)

In [7]:
from operator import truediv as div

def NNLSFrob(X, cols):
    """ Compute H, the coefficient matrix, by nonnegative least squares to minimize
    the Frobenius norm.  Given the data matrix X and the columns cols, H is
             \arg\min_{Y \ge 0} \| X - X(:, cols) H \|_F.
    Args:
        X: The data matrix.
        cols: The column indices.
    Returns:
        The matrix H and the relative resiual.
    """
    ncols = X.shape[1]
    H = np.zeros((len(cols), ncols))
    for i in xrange(ncols):
        sol, res = optimize.nnls(X[:, cols], X[:, i])
        H[:, i] = sol
    rel_res = np.linalg.norm(X - np.dot(X[:, cols], H), 'fro')
    rel_res /= np.linalg.norm(X, 'fro')
    return H, rel_res

def norm_axis(X, p=None, axis=None):
    """
    Compute entry-wise norms (! not induced/operator norms).
    :param X: The input matrix.
    :type X: :class:`scipy.sparse` of format csr, csc, coo, bsr, dok, lil, dia
    or :class:`numpy.matrix`
    :param p: Order of the norm.
    :type p: `str` or `float`
    """
    assert 1 in X.shape or p != 2 or axis is not None,\
        "Computing entry-wise norms only."

    n = np.linalg.norm(np.mat(X), ord=p, axis=axis)
    if axis is None:
        return n
    else:
        return np.array(n)
    
def elop(X, Y, op):
    """
    Compute element-wise operation of matrix :param:`X` and matrix :param:`Y`.
    :param X: First input matrix.
    :type X: :class:`scipy.sparse` of format csr, csc, coo, bsr, dok, lil, dia
    or :class:`numpy.matrix`
    :param Y: Second input matrix.
    :type Y: :class:`scipy.sparse` of format csr, csc, coo, bsr, dok, lil, dia
    or :class:`numpy.matrix`
    :param op: Operation to be performed.
    :type op: `func`
    """
    try:
        zp1 = op(1, 0)
        zp2 = op(0, 0)
        zp = zp1 != 0 or zp2 != 0
    except:
        zp = 0

    try:
        X[X == 0] = np.finfo(X.dtype).eps
        Y[Y == 0] = np.finfo(Y.dtype).eps
    except ValueError:
        return op(np.mat(X), np.mat(Y))
    return op(np.mat(X), np.mat(Y))

class SepNMF():
    
    def __init__(self, V, rank=30, n_run = 10, eps=0.005, oversampling_param = 5, n_power_iter=0,
                 min_comp=20):
        
        self.rank = rank
        self.n_run = n_run
        self.eps = eps
        self.oversampling_param = oversampling_param
        self.n_power_iter = n_power_iter
        self.min_comp = min_comp
        self.V = V
        
    def qr(self):
        """
        QR compression algorithm
        """
        q, _ = np.linalg.qr(self.V)
        return q.T.dot(self.V)

    def algo42(self):

        q = algo42(self.V, self.eps, self.rank)
        return q.T.dot(self.V)

    def count_gauss(self):
        """
        Project the columns of the matrix V into the lower dimension
        new_dim using count sketch + gaussian algorithm
        """

        old_dim = V.shape[0]

        # ksq = new_dim * new_dim
        ksq = self.rank * self.oversampling_param  # was not converging for scree plots of damle/sun

        R = np.random.randint(ksq, size=old_dim)
        C = np.arange(old_dim)
        D = np.random.choice([-1, 1], size=old_dim)
        S = scipy.sparse.csr_matrix((D, (R, C)), shape=(ksq, old_dim))

        G = np.random.randn(self.rank, ksq)
        M_red = dot(G, dot(S, self.V))
        return M_red.todense() / np.sqrt(self.rank)  
    
    def structured(self):
        """
        Structured compression algorithm
        """
        n = self.V.shape[1]
        comp_level = min(max(self.min_comp, self.rank + self.oversampling_param), n)
        omega = np.random.standard_normal(size=(n, comp_level))

        mat_h = self.V.dot(omega)
        for _ in range(n_power_iter):
            mat_h = self.V.dot(self.V.T.dot(mat_h))
        q, _ = np.linalg.qr(mat_h)
        return q.T.dot(self.V)
    
    def xray(self, V):
        """
        X-ray algorithm for extreme column selection in separable NMF.
        :param V: The data matrix.
        :type V: Instance of the :class:`scipy.sparse` sparse matrices
        types, :class:`numpy.ndarray`, or :class:`numpy.matrix`.
        :return: The indices of the columns chosen by X-ray.
        :rtype: List of ints.
        """
        cols = []
        R = V
        while len(cols) < self.rank:
            # Loop until we choose a column that has not been selected.
            while True:
                p = np.random.random((1, V.shape[0]))
                scores = norm_axis(np.dot(R.T, V), axis=0)
                scores = elop(scores, np.dot(p, V), div)
                scores[0, cols] = -1  # IMPORTANT
                best_col = np.argmax(scores)
                if best_col in cols:
                    # Re-try
                    continue
                else:
                    cols.append(best_col)
                    H = bpp(V[:, cols], V)
                    R = V - np.dot(V[:, cols], H)
                    break
        return cols
        
    
    def factorize(self):
        
        """
        Compute matrix factorization.
        Return fitted factorization model.
        """
        L, _ = np.linalg.qr(self.V)
        print(np.sum(L,0))
        R, _ = np.linalg.qr(self.V.T)
        print(np.sum(R,0))
        V = self.qr()
        print(np.sum(V,0))
        
        cols = self.xray(V)
        
        self.H = np.random.rand(self.rank, V.shape[1])
        self.W = self.V[:, cols]
        print(self.W.shape)
        
        for run in range(self.n_run):
            
            self.H = bpp(self.W, self.V)
            self.W = bpp(self.H.T, self.V.T).T
        
        print(np.sum(self.W,0))
        print(np.sum(self.H,0))
        #X = L.dot(self.W)
        #Y = self.H.dot(R)
            
        return self.W, self.H

In [8]:
sepnmf = SepNMF(rank=30, V = M.todense(), n_run = 1, eps=0.005, oversampling_param = 5, n_power_iter=3,
                 min_comp=20)

In [9]:
X, Y = sepnmf.factorize()

[[-8.88137526 -6.97621789 -1.         ...,  0.44835246  0.63695633
   2.83767204]]
[[-2.78226727 -2.44487543 -0.18815881 ..., -0.24159281  0.92704619 -1.        ]]
[[ -55688.91343987  -58860.15502865   -2953.3333333  ...,   -4833.56240869
    -2316.46906498 -142404.84673096]]
(14822, 30)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 1.57771163  0.          0.         ...,  0.          0.          0.        ]


In [13]:
np.where(X > 0)

(array([], dtype=int64), array([], dtype=int64))

In [14]:
np.where(Y > 0)

(array([ 7,  8,  9, 10, 18, 24]), array([0, 0, 0, 0, 0, 0]))

In [15]:
X

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [11]:
XY = X.dot(Y)

In [12]:
np.linalg.norm(M.todense() - XY)/np.linalg.norm(M.todense())

1.0

In [13]:
np.linalg.norm(M.todense() - XY)

1228035.0817721039

In [14]:
np.linalg.norm(M.todense())

1228035.0817721039

In [15]:
X.sum(0)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.])

In [27]:
list(filter(lambda x: x>0,Y.sum(0)))

[0.70376258897610477]